<a href="https://colab.research.google.com/github/kimhalyn/OPEN-API/blob/main/hourrf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 라이브러리
# colab에 xmltodict 라이브러리가 기본 설치되지 않으므로 별도 설치(외부 라이브러리)
!pip install xmltodict 
from urllib.request import urlopen, Request
from urllib.parse import urlencode, unquote, quote_plus
from bs4 import BeautifulSoup
import pandas as pd
import requests, xmltodict, json
from xml.etree.ElementTree import Element, SubElement, ElementTree
import xml.etree.ElementTree as ET
import pickle
import numpy as np

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import requests, bs4
import pandas as pd
from lxml import html
from urllib.request import Request, urlopen
from urllib.parse import urlencode, quote_plus, unquote
from time import sleep
import csv

url = 'http://opendata.kwater.or.kr/openapi-data/service/pubd/dam/excllncobsrvt/hourrf/list' # !!확인
key = 'jkd4PnVjssNP+g2WeRzs8lgc4SOelLsncvuMr81/9R9nSYKkiQAcb2xt8GTYgTJSQGQ4+htuRuyIXSwRXr210w==' # !!확인
 
api_key = requests.utils.unquote(key) #디코딩

# 댐코드 및 우량 관측소 코드 불러오기
data1 = pd.read_csv('/content/drive/MyDrive/water_resource_csv/damCode.csv') # !!!확인
data2 = pd.read_csv('/content/drive/MyDrive/water_resource_csv/excllcode.csv') # !!확인

# 댐코드 및 우량 관측소 코드 조회
damcode = data1.iloc[0:5, 1]
excllcode = data2.iloc[0:5, 2]

xmlrows = []

for code1 in damcode:
  for code2 in excllcode:

    Params = '?' + urlencode({quote_plus('ServiceKey') : api_key,
                                  quote_plus('sdate') : "2020-01-01",
                                  quote_plus('stime') : "00",
                                  quote_plus('edate') : "2020-01-01",
                                  quote_plus('etime') : "24",
                                  quote_plus('damcode') : code1, #댐코드
                                  quote_plus('excll') : code2, #우량관측소 코드
                                  quote_plus('numOfRows') : "800000",
                                  quote_plus('pageNo') : "1"})
    
#  print(Params)
  
  res = requests.get(url + Params).text.encode('utf-8') # 요청에 대한 응답내용(.encode빼기도 함)
  sleep(0.5)
  print(res) 
  
  xmlobj = bs4.BeautifulSoup(res, 'lxml') #lxml 파서(구문분석)  'lxml-xml'
  print(xmlobj)
  
  #item 태그 분리
  rows = xmlobj.findAll('item') # 매치되는 문자열을 리스트로 리턴
  xmlrows.append(rows)

print (xmlrows)


b'<?xml version="1.0" encoding="UTF-8" standalone="yes"?><response><header><resultCode>00</resultCode><resultMsg>NORMAL SERVICE.</resultMsg></header><body><items><item><acmtlrf>0</acmtlrf><hourrf>0</hourrf><no>24</no><obsrdt>2020-01-01 01:00</obsrdt></item><item><acmtlrf>0</acmtlrf><hourrf>0</hourrf><no>23</no><obsrdt>2020-01-01 02:00</obsrdt></item><item><acmtlrf>0</acmtlrf><hourrf>0</hourrf><no>22</no><obsrdt>2020-01-01 03:00</obsrdt></item><item><acmtlrf>0</acmtlrf><hourrf>0</hourrf><no>21</no><obsrdt>2020-01-01 04:00</obsrdt></item><item><acmtlrf>0</acmtlrf><hourrf>0</hourrf><no>20</no><obsrdt>2020-01-01 05:00</obsrdt></item><item><acmtlrf>0</acmtlrf><hourrf>0</hourrf><no>19</no><obsrdt>2020-01-01 06:00</obsrdt></item><item><acmtlrf>0</acmtlrf><hourrf>0</hourrf><no>18</no><obsrdt>2020-01-01 07:00</obsrdt></item><item><acmtlrf>0</acmtlrf><hourrf>0</hourrf><no>17</no><obsrdt>2020-01-01 08:00</obsrdt></item><item><acmtlrf>0</acmtlrf><hourrf>0</hourrf><no>16</no><obsrdt>2020-01-01 09:0

In [ ]:

filePath = '/content/drive/MyDrive/water_resource_csv/xmlrows.xml'


with open(filePath, 'wb') as lf:
    pickle.dump(xmlrows, lf)


with open(filePath, 'rb') as lf:
    readList = pickle.load(lf)


In [ ]:
# 컬럼 생성
rowList = []
nameList = []
columnList = []
xmlrowsLen = len(xmlrows)
target = 0
for i in range(0, xmlrowsLen):
    
   
    if len(xmlrows[i]) == 0:
       target = target + 1
      
    else:
        break 
columns = xmlrows[target][0].find_all()
columnsLen = len(columns)
for i in range(0, columnsLen):
    nameList.append(columns[i].name)
nameList = ["Damcode"] + ["Excllncobsrvtcode"] + nameList


In [ ]:
print(columns)
print(nameList)

[<acmtlrf>0</acmtlrf>, <hourrf>0</hourrf>, <no>24</no>, <obsrdt>2020-01-01 01:00</obsrdt>]
['Damcode', 'Excllncobsrvtcode', 'acmtlrf', 'hourrf', 'no', 'obsrdt']


In [ ]:
xmlrows[0]

[<item><acmtlrf>0</acmtlrf><hourrf>0</hourrf><no>24</no><obsrdt>2020-01-01 01:00</obsrdt></item>,
 <item><acmtlrf>0</acmtlrf><hourrf>0</hourrf><no>23</no><obsrdt>2020-01-01 02:00</obsrdt></item>,
 <item><acmtlrf>0</acmtlrf><hourrf>0</hourrf><no>22</no><obsrdt>2020-01-01 03:00</obsrdt></item>,
 <item><acmtlrf>0</acmtlrf><hourrf>0</hourrf><no>21</no><obsrdt>2020-01-01 04:00</obsrdt></item>,
 <item><acmtlrf>0</acmtlrf><hourrf>0</hourrf><no>20</no><obsrdt>2020-01-01 05:00</obsrdt></item>,
 <item><acmtlrf>0</acmtlrf><hourrf>0</hourrf><no>19</no><obsrdt>2020-01-01 06:00</obsrdt></item>,
 <item><acmtlrf>0</acmtlrf><hourrf>0</hourrf><no>18</no><obsrdt>2020-01-01 07:00</obsrdt></item>,
 <item><acmtlrf>0</acmtlrf><hourrf>0</hourrf><no>17</no><obsrdt>2020-01-01 08:00</obsrdt></item>,
 <item><acmtlrf>0</acmtlrf><hourrf>0</hourrf><no>16</no><obsrdt>2020-01-01 09:00</obsrdt></item>,
 <item><acmtlrf>0</acmtlrf><hourrf>0</hourrf><no>15</no><obsrdt>2020-01-01 10:00</obsrdt></item>,
 <item><acmtlrf>0</a

In [ ]:
total = []
damcodeLen = len(damcode)
excllcodeLen = len(excllcode)


def el_code(a): 
    columnList = []
    eachrows = xmlrows[a]
    eachrowslen = len(eachrows)
    for i in range(0, eachrowslen):
        columns = eachrows[i].find_all()
        columnsLen = len(columns)
        for j in range(0, columnsLen):
            eachColumn = columns[j].text 
            columnList.append(eachColumn)
        columnList = [damcode[a]] + [excllcode[a]] + columnList
        rowList.append(columnList)
        columnList = []
    return rowList



for a in range(0, damcodeLen, excllcodeLen):
    rowList=el_code(a)
    total.append(rowList)

result = pd.DataFrame(total[0], columns=nameList)

   

In [ ]:
print(damcodeLen)

5


In [ ]:
result.to_csv("/content/drive/MyDrive/water_resource_csv/hourrf_test1.csv",encoding="utf-8-sig")

In [ ]:
result

,Damcode,Excllncobsrvtcode,acmtlrf,hourrf,no,obsrdt
0,2403201,9000232,0,0,24,2020-01-01 01:00
1,2403201,9000232,0,0,23,2020-01-01 02:00
2,2403201,9000232,0,0,22,2020-01-01 03:00
3,2403201,9000232,0,0,21,2020-01-01 04:00
4,2403201,9000232,0,0,20,2020-01-01 05:00
5,2403201,9000232,0,0,19,2020-01-01 06:00
6,2403201,9000232,0,0,18,2020-01-01 07:00
7,2403201,9000232,0,0,17,2020-01-01 08:00
8,2403201,9000232,0,0,16,2020-01-01 09:00
9,2403201,9000232,0,0,15,2020-01-01 10:00
